In [43]:
#Lakshminarayanan
import pprint
import pandas as pd
import numpy as np
import pickle
import joblib

# Loading processed_corpus from pickle

**processed_pickle has tokenized value of song_data_with_genre['text'], which as given below**

song_data_with_genre['text'] = song_data_with_genre['title'] + ' ' + song_data_with_genre['artist_name'] + ' ' + song_data_with_genre['genre'] + ' ' + song_data_with_genre['release']

In [22]:
processed_corpus = pickle.load( open( "processed_corpus.pkl", "rb" ) )

# Using corpora dictionary from gensim 

In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)

In [24]:
len(dictionary.token2id)

43862

In [25]:
# testing with new doc
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(3005, 1), (7351, 1)]


# Loading bow_corpus from pickle

In [26]:
bow_corpus = pickle.load( open( "bow_corpus.pkl", "rb" ) )

In [42]:
tfidf = joblib.load('gensim_tfidf_model.pkl')

# USING SparseMatrixSimilarity FROM GENSIM

In [28]:
# Compute similarity against a corpus of documents by storing the sparse index matrix in memory. 
# The similarity measure used in cosine is between two vectors.

from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=len(dictionary.token2id))

In [29]:
query_document = "Systems Engineering".lower().split()
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]

In [30]:
doc_l = []
score_l = []
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    doc_l.append(document_number)
    score_l.append(score)
    

In [40]:
recommended_songs = pd.DataFrame(score_l,doc_l).head(10)
recommended_songs=recommended_songs.reset_index()
recommended_songs.columns = ['index','score']
recommended_songs.set_index('index',inplace = True)
recommended_songs

,score
index,
102576,0.515689
168317,0.434985
7166,0.426236
134317,0.425991
60802,0.395005
52044,0.385815
109564,0.379896
38726,0.378609
136961,0.378119


In [32]:
len(sims)

185184

In [33]:
sims[document_number]

0.0

In [34]:
song_data_with_genre=pd.read_csv('song_data_with_genre.csv')

In [35]:
song_data_with_genre = song_data_with_genre.drop(columns=['Unnamed: 0','Unnamed: 0.1','tid'])

In [36]:
song_data_with_genre

,title,artist_name,genre,song_id,release,year,artist_gender
0,El hijo del pueblo,Jorge Negrete,Latin,SOLJTLX12AB01890ED,32 Grandes Éxitos CD 2,1997,female
1,N Gana,Waldemar Bastos,World,SOGPCJI12A8C13CCA0,Afropea 3 - Telling Stories To The Sea,0,male
2,006,Lena Philipsson,Pop,SOSDCFG12AB0184647,Lena 20 År,1998,male
3,Rock-N-Rule,Emery,Rock,SOIMMJJ12AF72AD643,I'm Only A Man (Bonus Track Version),2007,female
4,I'm Ready,Aerosmith,Rock,SOEEHEY12CF5F88FB4,Honkin' On Bobo,2004,female
...,...,...,...,...,...,...,...
185179,Interrotte Speranze,Coronatus,Metal,SOIQNVK12A8C13B061,Lux Noctis,2007,male
185180,Wouldn't Get Far,The Game,Rap,SOQDFGV12AB0183F25,R&B Collaborations 2007,0,female
185181,Wouldn't Get Far,The Game,Rap,SOYKHSL12AF72A35AC,Wouldn't Get Far,0,female
185182,Chill,Jazz Addixx,Rap,SOIDAMG12AB0182B3A,Oxygen Refreshed (Instrumentals),0,female


In [37]:
d = []
for idx in top_10_reco.index:
    dic = dict(song_data_with_genre.loc[idx][['title','artist_name']])
    d.append(dic)

In [38]:
pd.DataFrame(d)

,title,artist_name
0,Faltered,Adelyn
1,UXB,Raven
2,Diamonds,Something With Numbers
3,Lavenderloops,Phonophani
4,Systems,Jah Free
5,Crazy World,Raven
6,Fire Power,Raven
7,End Of All Things,Phonophani
8,All For One,Raven
9,Take Control,Raven
